In [1]:
!pip install mediapipe
!pip install dollarpy

In [2]:
import mediapipe as mp
import cv2
import csv
from dollarpy import Point, Recognizer , Template
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:
def getPoints(videoPath) :
  left_leg      =[]
  right_leg     =[]
  
  face          =[]
  #left_wrist    =[]
  #right_wrist   =[]

  
  left_arm    =[]
  right_arm   =[]
  
  left_shoulder =[]
  right_shoulder=[]

  #left_elbow    =[]
  #right_elbow   =[]

  #left_hip      =[]
  #right_hip     =[]



  points        =[]
  with mp_holistic.Holistic(static_image_mode=True, min_detection_confidence = 0.7) as Holistic:
    video = cv2.VideoCapture(videoPath)
    while video.isOpened():
      ret , frame = video.read()
      if ret == True:
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = Holistic.process(cv2.flip(image,1))
        if results.pose_landmarks:
          #leg includes ankle,knee and hip
          #left_leg.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ANKLE].y,2))
          #left_leg.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_KNEE].y,2))
          #left_leg.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_HIP].y,2))
          
          face.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EAR].y,2))
          face.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y,2))
          #arm includes shoulder, elbow and wrist
          #left_arm.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y,2))
          #left_arm.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y,2))
          #left_arm.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y,2))

          #right_arm.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y,2))
          #right_arm.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y,2))
          #right_arm.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y,2))


          #right_leg.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ANKLE].y,2))
          #right_leg.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_KNEE].y,2))
          #right_leg.append(Point(results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].x,results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_HIP].y,2))

      else:
        break
    points = face
    return points

In [5]:
classes = []

In [7]:
vid = "./Poses/Test2.mkv"
pts = getPoints(vid)

import time
start = time.time()
reco = Recognizer(classes)
predicted = reco.recognize(pts)
print(start)
print(predicted)

1669053215.8985927
(None, 0)


In [1]:
!pip install imutils

In [2]:
# import the necessary packages
from collections import deque
from imutils.video import VideoStream
import numpy as np
import argparse
import cv2
import imutils
import time


In [4]:
# construct the argument parse and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-v", "--video",
	help="path to the (optional) video file")
ap.add_argument("-b", "--buffer", type=int, default=64,
	help="max buffer size")
args = vars(ap.parse_args())
# define the lower and upper boundaries of the "green"
# ball in the HSV color space, then initialize the
# list of tracked points
ColorLower = (0,0,0)
ColorUpper = (235, 235, 235)
pts = deque(maxlen=args["buffer"])
# if a video path was not supplied, grab the reference
# to the webcam
if not args.get("video", False):
	vs = VideoStream(src=0).start()
# otherwise, grab a reference to the video file
else:
	vs = cv2.VideoCapture(args["video"])
# allow the camera or video file to warm up
time.sleep(2.0)
# keep looping
while True:
	# grab the current frame
	frame = vs.read()
	# handle the frame from VideoCapture or VideoStream
	frame = frame[1] if args.get("video", False) else frame
	# if we are viewing a video and we did not grab a frame,
	# then we have reached the end of the video
	if frame is None:
		break
	# resize the frame, blur it, and convert it to the HSV
	# color space
	frame = imutils.resize(frame, width=600)
	blurred = cv2.GaussianBlur(frame, (11, 11), 0)
	hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
	# construct a mask for the color "green", then perform
	# a series of dilations and erosions to remove any small
	# blobs left in the mask
	mask = cv2.inRange(hsv, ColorLower, ColorUpper)
	mask = cv2.erode(mask, None, iterations=2)
	mask = cv2.dilate(mask, None, iterations=2)
  # find contours in the mask and initialize the current
	# (x, y) center of the ball
	cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
		cv2.CHAIN_APPROX_SIMPLE)
	cnts = imutils.grab_contours(cnts)
	center = None
	# only proceed if at least one contour was found
	if len(cnts) > 0:
		# find the largest contour in the mask, then use
		# it to compute the minimum enclosing circle and
		# centroid
		c = max(cnts, key=cv2.contourArea)
		((x, y), radius) = cv2.minEnclosingCircle(c)
		M = cv2.moments(c)
		center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
		# only proceed if the radius meets a minimum size
		if radius > 10:
			# draw the circle and centroid on the frame,
			# then update the list of tracked points
			cv2.circle(frame, (int(x), int(y)), int(radius),
				(0, 255, 255), 2)
			cv2.circle(frame, center, 5, (0, 0, 255), -1)
	# update the points queue
	pts.appendleft(center)
  # loop over the set of tracked points
	for i in range(1, len(pts)):
		# if either of the tracked points are None, ignore
		# them
		if pts[i - 1] is None or pts[i] is None:
			continue
		# otherwise, compute the thickness of the line and
		# draw the connecting lines
		thickness = int(np.sqrt(args["buffer"] / float(i + 1)) * 2.5)
		cv2.line(frame, pts[i - 1], pts[i], (0, 0, 255), thickness)
	# show the frame to our screen
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
	# if the 'q' key is pressed, stop the loop
	if key == ord("q"):
		break
# if we are not using a video file, stop the camera video stream
if not args.get("video", False):
	vs.stop()
# otherwise, release the camera
else:
	vs.release()
# close all windows
cv2.destroyAllWindows()

usage: ipykernel_launcher.py [-h] [-v VIDEO] [-b BUFFER]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\dell\AppData\Roaming\jupyter\runtime\kernel-7e5bed94-6c80-4c6a-90a0-64c949e2bc0e.json


SystemExit: 2

D:\software\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
